Imports

In [30]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from PIL import Image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split



Get Image Size


In [31]:
def get_sample_image_size(dataset_dir):
    """
    dataset_dir: path to something like "Dataset/training"
                 which has subfolders (e.g., glioma_tumor, meningioma_tumor, etc.)
    returns (width, height) of the first found image
    """
    # List subfolders (the class folders)
    class_folders = [f for f in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, f))]
    
    # We assume at least one class folder and at least one image
    first_class_folder = os.path.join(dataset_dir, class_folders[0])
    image_files = [f for f in os.listdir(first_class_folder) 
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    first_image_path = os.path.join(first_class_folder, image_files[0])
    
    # Open the image and check size
    with Image.open(first_image_path) as img:
        width, height = img.size
        print(f"Discovered image size: {width}x{height}")
        return width, height

# Example usage:
dataset_dir = "Dataset/testing"
detected_width, detected_height = get_sample_image_size(dataset_dir)


Discovered image size: 495x619


The Model

In [32]:
class LeNet5_512(nn.Module):
    def __init__(self, num_classes=4):
        super(LeNet5_512, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool = nn.AvgPool2d(kernel_size=2)

        # 16×125×125 = 250,000 after the second pooling
        self.fc1 = nn.Linear(16 * 125 * 125, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        # (3×512×512) => conv1 => (6×508×508) => pool => (6×254×254)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        # => conv2 => (16×250×250) => pool => (16×125×125)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        # Flatten => (batch_size, 250,000)
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

DataLoader

In [33]:

# Example normalization (commonly used for color images, same as ImageNet)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_test_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

training_data_path = "Dataset/training"
testing_data_path = "Dataset/testing"

# Full training dataset (with train augmentation)
full_train_dataset = datasets.ImageFolder(root=training_data_path, transform=train_transform)

# Split into train/val (80/20)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Apply simpler transform to validation set
val_dataset.dataset.transform = val_test_transform

# Test set
test_dataset = datasets.ImageFolder(root=testing_data_path, transform=val_test_transform)

batch_size = 8  # You may need to reduce batch_size if memory is an issue
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

class_names = full_train_dataset.classes
print("Classes:", class_names)
print("Training samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))
print("Testing samples:", len(test_dataset))


Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
Training samples: 2296
Validation samples: 574
Testing samples: 394


Training Function

In [35]:
def train_model(model, train_loader, val_loader, num_epochs=10, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        #################
        # Training Phase
        #################
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)   # shape: (batch_size, 4)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, dim=1)
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels).item()
            total_samples += labels.size(0)

        train_epoch_loss = running_loss / total_samples
        train_epoch_acc = running_corrects / total_samples

        ###################
        # Validation Phase
        ###################
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
        val_total_samples = 0

        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images = val_images.to(device)
                val_labels = val_labels.to(device)

                val_outputs = model(val_images)
                val_loss = criterion(val_outputs, val_labels)

                _, val_preds = torch.max(val_outputs, dim=1)
                val_running_loss += val_loss.item() * val_images.size(0)
                val_running_corrects += torch.sum(val_preds == val_labels).item()
                val_total_samples += val_labels.size(0)

        val_epoch_loss = val_running_loss / val_total_samples
        val_epoch_acc = val_running_corrects / val_total_samples

        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_acc:.4f} | "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

    print("Training complete.")
    return model

Training

In [ ]:
if __name__ == "__main__":
    # Initialize our LeNet5_512 model
    model = LeNet5_512(num_classes=4)

    # Train the model
    trained_model = train_model(model, train_loader, val_loader, num_epochs=5, lr=0.001)

    # Evaluate on the test set
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trained_model.eval()
    trained_model.to(device)

    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = trained_model(images)  # shape: (batch_size, 4)
            _, preds = torch.max(outputs, dim=1)

            test_correct += torch.sum(preds == labels).item()
            test_total += labels.size(0)

    test_accuracy = test_correct / test_total
    print(f"Test Accuracy: {test_accuracy:.4f}")

    # Save model weights (optional)
    torch.save(trained_model.state_dict(), "brain_tumor_lenet_512.pth")
